<a href="https://colab.research.google.com/github/485585r/GISPROJECTS/blob/main/barelandbuiltup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This AI is a demonstration of google earth engine's "smileRandomForest" classification AI model's potential when applied to a high resolution dataset. For this example, it's labelling builtup and bareland in Santa Maria.

In [ ]:
#!pip install geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.0 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11123 sha256=8ee47a0e2f8168b0b7eef747105d6ca16479ba25eaf3b83dc95e270c375faff8
  Stored in directory: /root/.cache/pip/wheels/04/24/fe/140a94a7f1036003ede94579e6b4227

In [3]:
import geemap
import ee
import collections
collections.Callable = collections.abc.Callable
ee.Authenticate()
ee.Initialize()

aoi= ee.FeatureCollection("projects/vpnww-299518/assets/oi")#Access via the assets tab in GEE(Table ID).
start_date="2018-01-01"#Set start_date(yy/mon/day)
end_date="2020-03-31"#Set End_date(yy/mon/day)

#-------------SENTINEL 1A(SAR) DATA------------------#
dataset = ee.ImageCollection('USDA/NAIP/DOQQ') \
          .filter(ee.Filter.date('2017-01-01', '2018-12-31')) \
          .filterBounds(aoi)
image = dataset.mosaic();
# Select bands for the classification
bands = ['R', 'G', 'B']

training = ee.FeatureCollection('projects/vpnww-299518/assets/yoTrainingAsset1GEE')
label = 'landcover'
Input=dataset.select(bands)
trainImage=image.sampleRegions(**{
    'collection':training,
    'properties':[label],
    'scale':30
})
trainingData=trainImage.randomColumn()

trainSet=trainingData.filter(ee.Filter.lessThan('random',0.75))
testdata=trainingData.filter(ee.Filter.greaterThanOrEquals('random',0.75))#visualization parameters applied on the Random forest classifier
init_params = {"numberOfTrees":400, # the number of individual decision tree models
              "variablesPerSplit":None,  # the number of features to use per split
              "minLeafPopulation":1, # smallest sample size possible per leaf
              "bagFraction":0.5, # fraction of data to include for each individual tree model
              "maxNodes":None, # max number of leafs/nodes per tree
               "seed":0}  # random seed for "random" choices like sampling. Setting this allows others to reproduce your exact results even with stocastic parameters

classifier = ee.Classifier.smileRandomForest(**init_params).train(trainSet, label, bands)
# Classify the image.

#Application of the random forest classifier for the purpose of image classification.
classified=image.classify(classifier)
palette = [
  'red', #builtup(0)
  'lightgreen', #unbuilt(1)#90ee90
 ];
Map=geemap.Map()
Map.addLayer(classified,
             {'min': 0, 'max': 1, 'palette': palette},
             'classification')
Map.centerObject(aoi, 14)
Map.addLayer(image)

Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=HFNpeSkCl4-2ATSdtCnLYGTniEamhGVNRKdMRY8oKMA&tc=d12HIJtA3RITNlOezgxlLq1fRXF9o6yxkGmTcWrVhXs&cc=2SxHIObzMc9wncWX2xah-lzz0MkgtMImJqDengTq9u8

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BXTJkYJTVEhGXkuR6lgGB3B-EVEUsDWKh6kKo6pD8jN2CUq_p8vGKo

Successfully saved authorization token.


Map(center=[34.435238988345674, -118.50986953520363], controls=(WidgetControl(options=['position', 'transparen…